In [1]:
import sys
import pandas as pd
sys.path.append('..')
from common.base import session

In [2]:
data = session.execute(''' SELECT trip_id,make,model,trip_duration_minutes,distance FROM daily_trip;''').all()
df = pd.DataFrame(data, columns=['trip_id','make','model','trip_duration_minutes','distance_travelled'])
df = df.groupby(['trip_id','make','model']).sum()

In [3]:

df['trip_duration_minutes'] = df['trip_duration_minutes'].round(2)
df['distance_travelled'] = df['distance_travelled'].round(2)
df.head()

,,,trip_duration_minutes,distance_travelled
trip_id,make,model,,
00922df3be5a4589ab385d0c2da2dd81,Audi,Q3,48.28,380.97
01b8a24510cd4e4684d67b96369286e0,BMW,3 Series GT,10.50,84.58
0244f7bc747b41fba6fcd75444736621,Jeep,Compass,9.12,84.21
02c51e56cc484711b218d3d01196687a,Audi,Q3,98.43,909.56
039c14a9a7df4096bb6e9445136fdf77,Hyundai,Creta,121.07,1118.54
